# Rearranger

In [2]:
import pandas as pd
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import tkinter as tk
from tkinter import filedialog
from pathlib import Path
from functools import partial, wraps

In [ ]:
root = tk.Tk()
root.withdraw()

# Open dialog to choose the .txt file
file_type = [('Text File', '*.txt')]
default_dir = 'Input'
file_path = filedialog.askopenfilename(filetypes=file_type, initialdir=default_dir)

# Read the .txt file into a dataframe
raw_df = pd.read_csv(file_path, sep = "\t")

raw_df.head()


In [ ]:
# Remove unnecessary columns
for col in raw_df.columns:
    if "unnamed" in col.lower() or "prob" in col.lower():
        raw_df.drop(col, axis=1, inplace=True)

raw_df

In [ ]:
# Extract the first frame of the video to get a sample image

# Open dialog to choose the .mp4 file
file_type = [('Video File', '*.mp4')]
default_dir = 'Input'
file_path = filedialog.askopenfilename(filetypes=file_type, initialdir=default_dir)
video_P = Path(file_path)

cap = cv2.VideoCapture(file_path)
_, frame = cap.read()
cap.release()

# Export frame as a .png file
image_P = video_P.parent / (video_P.stem + ".png")
cv2.imwrite(str(image_P), frame)


In [5]:
def get_tanks(im):
    example_tanks = {}

    tank_names = ['Top Left', 'Top Right', 'Bottom Left']

    def tank_selector(im, tank_name):
        while True:
            selected_pixel = cv2.selectROI('Selecting tank at position'+str(tank_name), im)
            
            # print out the coordinates of top left and bottom right pixel of the ROI
            print('Tank at position: ', tank_name)
            print('Top left pixel: x = {}, y = {}'.format(selected_pixel[0], selected_pixel[1]))
            print('Bottom right pixel: x = {}, y = {}'.format(selected_pixel[0] + selected_pixel[2], selected_pixel[1] + selected_pixel[3]))
            print('Middle line: y = {}'.format(selected_pixel[1] + selected_pixel[3] / 2))
            tank_d = math.ceil((selected_pixel[2] + selected_pixel[3]) / 2)
            print('Diameter of the tank : ', tank_d)
            print()
            example_tanks[tank_name] = selected_pixel
            cv2.destroyAllWindows()
            break

    for name in tank_names:
        tank_selector(im, name)

    return example_tanks

In [3]:
image_P = Path(r"C:\Code\ChungyuanProjects\TowerAssayAnalyzer\Input\Zebrafish Tower\A - Novel Tank Test\01 - Control (1st Batch).png")


In [6]:

im = cv2.imread(str(image_P))
example_wells = get_tanks(im)

Tank at position:  Top Left
Top left pixel: x = 142, y = 9
Bottom right pixel: x = 283, y = 342
Middle line: y = 175.5
Diameter of the tank :  237

Tank at position:  Top Right
Top left pixel: x = 1035, y = 16
Bottom right pixel: x = 1180, y = 347
Middle line: y = 181.5
Diameter of the tank :  238

Tank at position:  Bottom Left
Top left pixel: x = 136, y = 379
Bottom right pixel: x = 280, y = 707
Middle line: y = 543.0
Diameter of the tank :  236



In [7]:
tanks_col = 5
tanks_row = 2

# tanks number
# 10 9 8 7 6
# 5 4 3 2 1

tanks_dict = {}

TL_num = tanks_col * tanks_row
TR_num = TL_num - tanks_col + 1
BL_num = tanks_col

tanks_dict[TL_num] = example_wells['Top Left']
tanks_dict[TR_num] = example_wells['Top Right']
tanks_dict[BL_num] = example_wells['Bottom Left']
tanks_dict[1] = (tanks_dict[6][0] - (tanks_dict[10][0] - tanks_dict[5][0]), tanks_dict[6][1] - (tanks_dict[10][1] - tanks_dict[5][1]), tanks_dict[6][2]*(tanks_dict[5][2]/tanks_dict[10][2]), tanks_dict[6][3]*(tanks_dict[5][3]/tanks_dict[10][3]))

def fill_row(tanks_dict, start, end, display = True):
    x_sep = (tanks_dict[start][0] - tanks_dict[end][0])/(end - start)
    y_sep = (tanks_dict[start][1] - tanks_dict[end][1])/(end - start)
    w_sep = (tanks_dict[start][2] - tanks_dict[end][2])/(end - start)
    h_sep = (tanks_dict[start][3] - tanks_dict[end][3])/(end - start)

    for i in range(start+1, end):
        tanks_dict[i] = (tanks_dict[end][0] + x_sep*(end-i), tanks_dict[end][1] + y_sep*(end-i), tanks_dict[end][2] + w_sep*(end-i), tanks_dict[end][3] + h_sep*(end-i))
    
    if display:
        for key, value in locals().items():
            print(key + ' = ' + str(value))

    return tanks_dict

for row in range(0, tanks_row):
    tanks_dict = fill_row(tanks_dict, 1 + row*tanks_col, (row+1)*tanks_col)
    # row = 0, fill(tanks_dict, 1 + 0*5,  (0+1)*5)
    # row = 1, fill(tanks_dict, 1 + 1*5,  (1+1)*5)

# tanks_dict = fill_row(tanks_dict, 1, 5)
# tanks_dict = fill_row(tanks_dict, 6, 10)

# round up all the values in tanks_dict to integers
for key in tanks_dict:
    tanks_dict[key] = tuple([int(round(i)) for i in tanks_dict[key]])

tanks_dict


tanks_dict = {10: (142, 9, 141, 333), 6: (1035, 16, 145, 331), 5: (136, 379, 144, 328), 1: (1029, 386, 148.08510638297872, 326.03003003003005), 2: (805.75, 384.25, 147.06382978723406, 326.52252252252254), 3: (582.5, 382.5, 146.04255319148936, 327.015015015015), 4: (359.25, 380.75, 145.02127659574467, 327.5075075075075)}
start = 1
end = 5
display = True
x_sep = 223.25
y_sep = 1.75
w_sep = 1.0212765957446805
h_sep = -0.49249249249248805
i = 4
tanks_dict = {10: (142, 9, 141, 333), 6: (1035, 16, 145, 331), 5: (136, 379, 144, 328), 1: (1029, 386, 148.08510638297872, 326.03003003003005), 2: (805.75, 384.25, 147.06382978723406, 326.52252252252254), 3: (582.5, 382.5, 146.04255319148936, 327.015015015015), 4: (359.25, 380.75, 145.02127659574467, 327.5075075075075), 7: (811.75, 14.25, 144.0, 331.5), 8: (588.5, 12.5, 143.0, 332.0), 9: (365.25, 10.75, 142.0, 332.5)}
start = 6
end = 10
display = True
x_sep = 223.25
y_sep = 1.75
w_sep = 1.0
h_sep = -0.5
i = 9


{10: (142, 9, 141, 333),
 6: (1035, 16, 145, 331),
 5: (136, 379, 144, 328),
 1: (1029, 386, 148, 326),
 2: (806, 384, 147, 327),
 3: (582, 382, 146, 327),
 4: (359, 381, 145, 328),
 7: (812, 14, 144, 332),
 8: (588, 12, 143, 332),
 9: (365, 11, 142, 332)}

In [8]:
im = cv2.imread(str(image_P))

# Display predicted tanks
def display_tanks():
    for i in range(1, 11):
        x, y, w, h = tanks_dict[i]
        cv2.rectangle(im, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(im, str(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow('Tanks', im)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

display_tanks()


In [9]:
import random
from Libs.misc import display_coords

def coord_generator(box, points = 1000):
    x, y, w, h = box
    coords = []
    for i in range(points):
        x_coord = random.randint(x, x+w)
        y_coord = random.randint(y, y+h)
        coords.append((x_coord, y_coord))
    return coords

def mix_up(input_dict, mix_ratio = 0.3):
    # input_dict has 10 keys 
    # keys are 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
    # values are coordinates inside the tanks
    # mix_ratio is the ratio of mixing up the coordinates
    # mix_ratio = 0.3 means 30% of the coordinates will be mixed up

    for i in range(1, len(input_dict)+1):
        if i == 1:
            continue
        else:
            for j in range(1, len(input_dict[i])+1):
                if random.random() < mix_ratio:
                    # swap the coordinates
                    input_dict[i][j-1], input_dict[i-1][j-1] = input_dict[i-1][j-1], input_dict[i][j-1]
    return input_dict
                    

test_dict = {}
for i in range(1, 11):
    test_dict[i] = coord_generator(tanks_dict[i])


columns = []
for i in range (1, 11):
    columns.append('x' + str(i))
    columns.append('y' + str(i))

test_df = pd.DataFrame(columns = columns)
for i in range(1, 11):
    test_df['x' + str(i)] = [x for x, y in test_dict[i]]
    test_df['y' + str(i)] = [y for x, y in test_dict[i]]

test_df

,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,x6,y6,x7,y7,x8,y8,x9,y9,x10,y10
0,1072,500,924,711,679,613,459,688,181,668,1163,227,937,106,673,86,426,300,216,214
1,1134,522,886,659,632,486,447,457,246,656,1176,167,923,24,602,224,381,101,189,166
2,1161,509,865,507,617,664,374,397,156,700,1121,178,818,188,705,156,441,229,273,219
3,1173,612,819,434,716,676,379,652,205,605,1125,131,943,63,687,210,457,104,195,117
4,1149,463,829,631,644,607,447,530,254,436,1140,85,952,311,595,253,435,33,162,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1160,684,926,711,682,509,408,655,191,668,1123,341,888,146,624,334,482,121,167,319
996,1101,485,893,646,690,456,500,613,268,608,1074,140,841,283,588,131,454,235,234,44
997,1075,410,833,437,684,388,483,697,194,474,1127,137,865,107,703,256,431,24,246,20
998,1104,493,953,708,716,395,385,550,184,569,1175,209,817,77,598,163,368,59,165,48


In [11]:
display_coords(test_df, im)

In [12]:
mixed_dict = mix_up(test_dict, mix_ratio = 0.3)
mixed_dict

# turn test_dict into dataframe, with 20 columns
# columns = ['x1', 'y1', 'x2', 'y2', ..., 'x10', 'y10']
# rows = 1000

import pandas as pd



mixed_df = pd.DataFrame(columns = columns)
for i in range(1, 11):
    mixed_df['x' + str(i)] = [x for x, y in mixed_dict[i]]
    mixed_df['y' + str(i)] = [y for x, y in mixed_dict[i]]

mixed_df

,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,x6,y6,x7,y7,x8,y8,x9,y9,x10,y10
0,1072,500,924,711,459,688,679,613,1163,227,181,668,937,106,426,300,673,86,216,214
1,886,659,632,486,1134,522,447,457,246,656,1176,167,602,224,923,24,189,166,381,101
2,1161,509,865,507,617,664,156,700,1121,178,374,397,705,156,818,188,273,219,441,229
3,819,434,716,676,1173,612,379,652,205,605,1125,131,687,210,457,104,195,117,943,63
4,1149,463,644,607,829,631,447,530,254,436,952,311,1140,85,595,253,435,33,162,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1160,684,682,509,408,655,191,668,1123,341,926,711,888,146,624,334,482,121,167,319
996,1101,485,893,646,690,456,500,613,1074,140,268,608,588,131,841,283,454,235,234,44
997,833,437,1075,410,684,388,194,474,1127,137,865,107,483,697,703,256,431,24,246,20
998,1104,493,953,708,716,395,385,550,1175,209,817,77,184,569,598,163,368,59,165,48


In [13]:
display_coords(mixed_df, im)

In [14]:
cv2.imshow('Tanks', im)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [15]:
def in_box(input_x, input_y, box):
    x, y, w, h = box
    return x <= input_x <= x + w and y <= input_y <= y + h
    
def in_which_tank(x, y):
    for tank_num, tank_box in tanks_dict.items():
        if in_box(x, y, tank_box):
            return tank_num
    return -1

true_df = pd.DataFrame(columns=test_df.columns)
for idx, row in test_df.iterrows():
    for j in range(1, 11):
        x = row['x' + str(j)]
        y = row['y' + str(j)]
        
        tank = in_which_tank(x, y)

        if tank != -1:
            true_df.loc[idx, 'x' + str(j)] = x
            true_df.loc[idx, 'y' + str(j)] = y     

true_df

,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,x6,y6,x7,y7,x8,y8,x9,y9,x10,y10
0,1072,500,924,711,679,613,459,688,181,668,1163,227,937,106,673,86,426,300,216,214
1,1134,522,886,659,632,486,447,457,246,656,1176,167,923,24,602,224,381,101,189,166
2,1161,509,865,507,617,664,374,397,156,700,1121,178,818,188,705,156,441,229,273,219
3,1173,612,819,434,716,676,379,652,205,605,1125,131,943,63,687,210,457,104,195,117
4,1149,463,829,631,644,607,447,530,254,436,1140,85,952,311,595,253,435,33,162,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1160,684,926,711,682,509,408,655,191,668,1123,341,888,146,624,334,482,121,167,319
996,1101,485,893,646,690,456,500,613,268,608,1074,140,841,283,588,131,454,235,234,44
997,1075,410,833,437,684,388,483,697,194,474,1127,137,865,107,703,256,431,24,246,20
998,1104,493,953,708,716,395,385,550,184,569,1175,209,817,77,598,163,368,59,165,48


In [16]:
display_coords(true_df, im)